In [35]:
from pyalgotrade import strategy
from pyalgotrade.broker import backtesting
from datetime import datetime
from oanda import fx_lib
import math
import json

class NU3B(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, order_books):
        super(NU3B, self).__init__(feed)
        commission = backtesting.TradePercentage(0)
        self._setBroker(backtesting.Broker(100000, feed, commission))
        self.getBroker().setCommission(backtesting.TradePercentage(0.1))
        self.setUseEventDateTimeInLogs(True)
        self.setDebugMode(True)

        self.__open_positions = set()
        self.__pending_positions = set()
        self.__instrument = instrument
        self.__order_books = order_books
        self.__spread_pips = 0.5
        self.__units = 100
        self.__sliding_minutes = 1

        # params
        self.__params = {
            "spreadAllowance": -1,
            "profitDistance": 9,
            "stopLossDistance": 5,
            "stopOrders": {0.3, 0.5}
        }

    def onEnterOk(self, position):
        self.__open_positions.add(position)
        self.__pending_positions.remove(position)

    def onExitOk(self, position):
        self.__open_positions.remove(position)

    def onBars(self, bars):
        if self.is_lambda_invoke_time() and self.is_restraint_time_zone():
            self.cancel_all_exist_orders()
            self.close_all_open_position()
        if self.is_lambda_invoke_time() and not self.is_restraint_time_zone():
            self.cancel_all_exist_orders()
            bar = bars[self.__instrument]
            now_price = bar.getClose()
            time_stamp_str = str(self.get_current_order_book_time_stamp())
            self.info(time_stamp_str)
            if not time_stamp_str in self.__order_books:
                return
            buckets = order_book[time_stamp_str]
            new_order_params = create_new_orders_logic_type_b(
                self.__instrument,
                self.__params,
                buckets,
                now_price,
                target_range=7)

            # execute orders
            for p in new_order_params:
                o = self.create_new_order(p["side"], self.__units, now_price, p["enterPrice"], p["limitPrice"], p["stopPrice"])
                self.__pending_positions.add(o)

    def get_current_order_book_time_stamp(self):
        return math.floor(self.getCurrentDateTime().timestamp()) - self.__sliding_minutes * 60

    def is_lambda_invoke_time(self):
        dt = math.floor(self.getCurrentDateTime().timestamp()) - (self.__sliding_minutes * 60)
        return fx_lib.is_order_book_update_time(datetime.fromtimestamp(dt))

    def is_restraint_time_zone(self):
        return self.getCurrentDateTime().hour == 21 or self.getCurrentDateTime().hour == 20

    def create_new_order(self, side: str, units:int, now_price:float, enter_price:float, limit_price:float, stop_price:float):
        self.info("oooooooooooo")
        spread_price = fx_lib.pips_to_price(self.__instrument, self.__spread_pips)
        half_spread = spread_price/2
        if side == "Ask":  # 買い
            if enter_price < now_price:
                position = self.enterLongLimit(self.__instrument, enter_price+half_spread, units)
                position.exitLimit(limit_price-half_spread)
                position.exitStop(stop_price-half_spread)
                return position
            else:
                position = self.enterLongStop(self.__instrument, enter_price+half_spread, units)
                position.exitLimit(limit_price-half_spread)
                position.exitStop(stop_price-half_spread)
                return position
        if side == "Bid":  # 売り
            if enter_price > now_price:
                position = self.enterShortLimit(self.__instrument, enter_price-half_spread, units)
                position.exitLimit(limit_price+half_spread)
                position.exitStop(stop_price+half_spread)
                return position
            else:
                position = self.enterShortStop(self.__instrument, enter_price-half_spread, units)
                position.exitLimit(limit_price+half_spread)
                position.exitStop(stop_price+half_spread)
                return position

    def cancel_all_exist_orders(self):
        for p in self.getActivePositions():
            if p.entryActive():
                p.cancelExit()
                p.cancelEntry()

    def close_all_open_position(self):
        for p in self.getActivePositions():
            if p.isOpen():
                p.exitMarket()

def create_new_orders_logic_type_b(instrument: str, params: dict, buckets: list, now_price: float, target_range: int):
    buckets = fx_lib.divide_buckets_up_and_down(buckets, now_price)
    long = buckets["long"]
    short = buckets["short"]

    # short order
    orders = []
    for i in range(target_range):
        activatable = True
        j = 0
        for lo in params["stopOrders"]:
            if long[i+j]["longCountPercent"] < lo:
                activatable = False
                break
            j += 1
        if activatable:
            enter_price = long[i]["price"] - fx_lib.pips_to_price(instrument, params["spreadAllowance"])
            limit_price = enter_price - fx_lib.pips_to_price(instrument, params["profitDistance"])
            stop_price = enter_price + fx_lib.pips_to_price(instrument, params["stopLossDistance"])
            order = {"side": "Bid", "enterPrice": enter_price, "limitPrice": limit_price, "stopPrice": stop_price}
            orders.append(order)
            break

    # long orders
    orders = []
    for i in range(target_range):
        activatable = True
        j = 0
        for so in params["stopOrders"]:
            if short[i+j]["shortCountPercent"] < so:
                activatable = False
                break
            j += 1
        if activatable:
            enter_price = short[i]["price"] + fx_lib.pips_to_price(instrument, params["spreadAllowance"])
            limit_price = enter_price + fx_lib.pips_to_price(instrument, params["profitDistance"])
            stop_price = enter_price - fx_lib.pips_to_price(instrument, params["stopLossDistance"])
            order = {"side": "Ask", "enterPrice": enter_price, "limitPrice": limit_price, "stopPrice": stop_price}
            orders.append(order)
            break

    return orders

In [36]:
from pyalgotrade import plotter
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
from pyalgotrade.stratanalyzer import returns
import time
feed = csvfeed.GenericBarFeed(frequency=5*Frequency.SECOND)
feed.addBarsFromCSV("USD_JPY", "../data/candles/USD_JPY_S5_2020.csv")
order_book_json = open("../data/order_book/USD_JPY_OB_2020.json")
order_book = json.load(order_book_json)
myStrategy = NU3B(feed, "USD_JPY", order_book)
returnsAnalyzer = returns.Returns()
myStrategy.attachAnalyzer(returnsAnalyzer)
plt = plotter.StrategyPlotter(myStrategy)
plt.getOrCreateSubplot("returns").addDataSeries("Simple returns", returnsAnalyzer.getReturns())

# run
start = time.time()
myStrategy.run()
elapsed_time = time.time() - start

myStrategy.info("Final portfolio value: $%.2f" % myStrategy.getResult())
plt.plot()


print(elapsed_time)

2020-01-01 23:01:00 strategy [INFO] 1577887200
2020-01-01 23:21:00 strategy [INFO] 1577888400
2020-01-01 23:41:00 strategy [INFO] 1577889600
2020-01-02 00:21:00 strategy [INFO] 1577892000
2020-01-02 01:01:00 strategy [INFO] 1577894400
2020-01-02 02:21:00 strategy [INFO] 1577899200
2020-01-02 04:01:00 strategy [INFO] 1577905200
2020-01-02 07:01:00 strategy [INFO] 1577916000
2020-01-02 07:21:00 strategy [INFO] 1577917200
2020-01-02 08:01:00 strategy [INFO] 1577919600
2020-01-02 08:21:00 strategy [INFO] 1577920800
2020-01-02 08:41:00 strategy [INFO] 1577922000
2020-01-02 09:01:00 strategy [INFO] 1577923200
2020-01-02 09:21:00 strategy [INFO] 1577924400
2020-01-02 10:21:00 strategy [INFO] 1577928000
2020-01-02 11:21:00 strategy [INFO] 1577931600
2020-01-02 13:01:00 strategy [INFO] 1577937600
2020-01-02 13:21:00 strategy [INFO] 1577938800
2020-01-02 14:41:00 strategy [INFO] 1577943600
2020-01-02 15:41:00 strategy [INFO] 1577947200
2020-01-02 16:01:00 strategy [INFO] 1577948400
2020-01-02 16

AssertionError: 